In [45]:
# Q1 Running Elastic
## What's the version.build_hash value?

In [55]:
!curl localhost:9200

{
  "name" : "cc9c59a29a78",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "zYwUVoiRTd2c0Z0MDmSJ1Q",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   539  100   539    0     0   8476      0 --:--:-- --:--:-- --:--:--  8555


In [56]:
# Q2 Indexing the data

In [57]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [58]:
## Which function do you use for adding your data to elastic?

#- insert
#- index
#- put
#- add

# answer: index

In [59]:
# Q3 Searching

In [60]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [61]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

es_client.info()

ObjectApiResponse({'name': 'cc9c59a29a78', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'zYwUVoiRTd2c0Z0MDmSJ1Q', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [62]:
# creating docker image for homework - terminal

#docker run -it \
#    --rm \
#    --name elasticsearch_hw \
#    -p 9200:9200 \
#    -p 9300:9300 \
#    -e "discovery.type=single-node" \
#    -e "xpack.security.enabled=false" \
#    docker.elastic.co/elasticsearch/elasticsearch:8.4.3

In [63]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

# very similar to minsearch, we are creating a new index, with the settings and mappings
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [64]:
from tqdm import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:50<00:00, 18.73it/s]


In [65]:
query = "How do I execute a command in a running docker container?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"], # we are giving more weight to the question field
                    "type": "best_fields" # we are looking for the best match
                }
            },
            # "filter": {
            #     "term": {
            #         "course": "data-engineering-zoomcamp"
            #     }
            # }
        }
    }
}


response = es_client.search(index=index_name, body=search_query)

In [66]:
for k,v in response['hits']['hits'][0].items():
    print(k, v)

_index course-questions
_id QFYNSpABmR4g_r5q2cSQ
_score 84.050095
_source {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}


In [67]:
#4 Filtering

In [68]:
search_query = {
    "size": 3, #
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"], # we are giving more weight to the question field
                    "type": "best_fields" # we are looking for the best match
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp" # we are filtering by course
                }
            }
        }
    }
}

In [69]:
response = es_client.search(index=index_name, body=search_query)

In [70]:
response['hits']['hits'][-1]

{'_index': 'course-questions',
 '_id': 'YFYNSpABmR4g_r5q38Tk',
 '_score': 49.938507,
 '_source': {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from a different folder into docker container’s working directory?',
  'course': 'machine-learning-zoomcamp'}}

In [71]:
# visualizing the response
response['hits']['hits'][0]['_source']

{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
 'section': '5. Deploying Machine Learning Models',
 'question': 'How do I debug a docker container?',
 'course': 'machine-learning-zoomcamp'}

In [72]:
context_template = """
Q: {question}
A: {text}
""".strip()

context = ""

for resp in response['hits']['hits']:
    question = resp['_source']['question']
    answer = resp['_source']['text']

    context = context + context_template.format(question=question, text=answer) + "\n\n"


print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [73]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [74]:
prompt = prompt_template.format(question=query, context=context)

print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [75]:
len(prompt.strip())

1462

In [77]:
#!pip install tiktoken

In [78]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

In [79]:
len(encoding.encode(prompt))

323

In [81]:
encoding.decode_single_token_bytes(63842)

b"You're"

In [ ]:
# Bonus

In [82]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [83]:
def llm(prompt):
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    ) 
    
    return response.choices[0].message.content

In [85]:
response_openAI = llm(prompt)

In [87]:
print(response_openAI)

To execute a command in a running Docker container, you can use the `docker exec` command. First, find the container ID of the running container using `docker ps`, and then use the `docker exec` command with the appropriate options. Here is the step-by-step process:

1. List the running Docker containers to find the container ID:
   ```bash
   docker ps
   ```

2. Execute a command in the specific container using its container ID. For example, to start a bash session in the container:
   ```bash
   docker exec -it <container-id> bash
   ```

Replace `<container-id>` with the actual ID of your running container.


In [89]:
#Suppose that on average per request we send 150 tokens and receive back 250 tokens.

#How much will it cost to run 1000 requests?

#You can see the prices here

#On June 17, the prices for gpt4o are:

#Input: $0.005 / 1K tokens
#Output: $0.015 / 1K tokens
#You can redo the calculations with the values you got in Q6 and Q7

# Prices for gpt4 on June 17
input_cost_per_1k_tokens = 0.005  # $0.005 per 1K tokens
output_cost_per_1k_tokens = 0.015  # $0.015 per 1K tokens

tokens_per_request = 150
tokens_per_response = 250

requests = 1000

# Calculate total tokens for requests and responses
total_input_tokens = tokens_per_request * requests
total_output_tokens = tokens_per_response * requests

# Calculate cost by dividing the total tokens by 1000 and then multiplying by the cost per 1K tokens
input_cost_total = (total_input_tokens / 1000) * input_cost_per_1k_tokens
output_cost_total = (total_output_tokens / 1000) * output_cost_per_1k_tokens

# Sum the input and output costs to get the total cost
total_cost = input_cost_total + output_cost_total

total_cost

4.5